In [1]:
import numpy as np
import json
import matplotlib.pyplot as plt
from cieluv import *
from RootPolyRegression import *
%matplotlib inline

In [2]:
patch_spectras = ['Dark Skin.reflect', 'Light Skin.reflect', 'Blue Sky.reflect',
                 'Foilage.reflect', 'Blue Flower.reflect', 'Bluish Green.reflect',
                 'Orange.reflect', 'Purplish Blue.reflect', 'Moderate Red.reflect',
                 'Purple.reflect', 'Yellow Green.reflect', 'Orange Yellow.reflect',
                 'Blue.reflect', 'Green.reflect', 'Red.reflect',
                 'Yellow.reflect', 'Magenta.reflect', 'Cyan.reflect',
                 'White.reflect', 'Neutral-8.reflect', 'Neutral-6.5.reflect',
                 'Neutral-5.reflect', 'Neutral-3.5.reflect', 'Black.reflect']

In [3]:
x = np.linspace(380, 780, 101)

#D65 source
src = np.loadtxt("./d65.illum")
x_src = np.linspace(300, 830, 531)
src = np.interp(x, x_src, src)
D65 = np.diag(src)

#XYZ spectral sensitivities
f = open('camera.kinect1.json', 'r')
xyz_dict = json.load(f)
XYZ_R = np.array(xyz_dict['sensitivities']['red'])[:, 1]
XYZ_G = np.array(xyz_dict['sensitivities']['green'])[:, 1]
XYZ_B = np.array(xyz_dict['sensitivities']['blue'])[:, 1]
x_XYZ = np.linspace(365, 830, 94) 
sens_XYZ_R = np.interp(x, x_XYZ, XYZ_R)
sens_XYZ_G = np.interp(x, x_XYZ, XYZ_G)
sens_XYZ_B = np.interp(x, x_XYZ, XYZ_B)

#RGB camera sensitivities
cam_R, cam_G, cam_B = np.loadtxt("cam_R.spect"), np.loadtxt("cam_G.spect"), np.loadtxt("cam_B.spect")
x_RGB = np.linspace(380, 784, 101)
sens_RGB_R = np.interp(x, x_RGB, cam_R)
sens_RGB_G = np.interp(x, x_RGB, cam_G)
sens_RGB_B = np.interp(x, x_RGB, cam_B)


In [4]:
X_WHITE = np.dot(sens_XYZ_R.T, src)
Y_WHITE = np.dot(sens_XYZ_G.T, src)
Z_WHITE = np.dot(sens_XYZ_B.T, src)
print("White point: ", X_WHITE, Y_WHITE, Z_WHITE)
WHITE_POINT_XYZ_D65 = [X_WHITE, Y_WHITE, Z_WHITE]
WHITE_POINT_XYZ_D65 /= Y_WHITE
WHITE_POINT_XYZ_D65

White point:  2754.907432227824 2905.2600332985335 3118.0681991097817


array([0.94824814, 1.        , 1.07324927])

In [5]:
colorsXYZ = []
for patch in patch_spectras:
    refl = np.loadtxt("./patches/" + patch)
    X = np.dot(np.dot(sens_XYZ_R.T, D65), refl)
    Y = np.dot(np.dot(sens_XYZ_G.T, D65), refl)
    Z = np.dot(np.dot(sens_XYZ_B.T, D65), refl)
    c = np.array([X, Y, Z]) / Y_WHITE
    colorsXYZ.append(c)
colorsXYZ = np.array(colorsXYZ)

colorsRGB = []
for patch in patch_spectras:
    refl = np.loadtxt("./patches/" + patch)
    R = np.dot(np.dot(sens_RGB_R.T, D65), refl)
    G = np.dot(np.dot(sens_RGB_G.T, D65), refl)
    B = np.dot(np.dot(sens_RGB_B.T, D65), refl)
    c = np.array([R, G, B]) / Y_WHITE
    colorsRGB.append(c)
colorsRGB = np.array(colorsRGB)

In [6]:
power = 3
R = colorsRGB.T
R = get_rp_features(R, power)
Q = colorsXYZ.T
print("R shape: ", R.shape)
print("Q shape: ", Q.shape)

y_r = Q[0]
y_g = Q[1]
y_b = Q[2]

R shape:  (13, 24)
Q shape:  (3, 24)


In [7]:
err = 0
for i in range(R.shape[1]):
    Q_train = np.concatenate((Q[:, :i], Q[:, (i+1):]), axis=1)
    R_train = np.concatenate((R[:, :i], R[:, (i+1):]), axis=1)
    Q_test = Q[:, i]
    R_test = R[:, i]
    A = np.dot(np.dot(Q_train, R_train.T), np.linalg.inv(np.dot(R_train, R_train.T)))
    print("[", i, "]", "========")
    train_err = 0
    for j in range(Q_train.shape[1]):
        deltaE = CIELUV_DeltaE_from_XYZ(np.dot(A, R_train[:, j]), Q_train[:, j], WHITE_POINT_XYZ_D65)
        print(deltaE, np.dot(A, R_train[:, j]), Q_train[:, j])
        train_err += deltaE
    print("Error on train:", train_err / 23)
    deltaE = CIELUV_DeltaE_from_XYZ(np.dot(A, R_test), Q_test, WHITE_POINT_XYZ_D65)
    print("Error on test:", deltaE)
    err += deltaE
    
print("!", err / 24)

[ 0 ] ========
0.5190729574805596 [0.34568997 0.32740862 0.24487872] [0.34470537 0.32552718 0.24414595]
1.6896965574538156 [0.16679041 0.18543218 0.32567191] [0.16858397 0.18470163 0.32331183]
0.8941482049236813 [0.09933656 0.12130451 0.07007371] [0.09753446 0.11862227 0.06664057]
0.8106377760684884 [0.23414588 0.23085903 0.41104435] [0.23298799 0.23042309 0.4132732 ]
0.1127266289813724 [0.30076537 0.40665615 0.41082224] [0.30047266 0.4062608  0.41098012]
1.2934006917524525 [0.34546041 0.27365794 0.07154489] [0.34395419 0.27394123 0.07284493]
0.5530628471259 [0.13192502 0.1266595  0.35475846] [0.13105796 0.1257724  0.35547934]
1.0784956303839728 [0.25996327 0.1867316  0.14578756] [0.26126758 0.18672926 0.14480596]
1.7680518101149219 [0.084523   0.06911049 0.15301811] [0.08626121 0.06988336 0.15088751]
0.8385351310678096 [0.32823507 0.40233713 0.11107855] [0.32794379 0.40382406 0.1121419 ]
0.5090697736098112 [0.42856125 0.3868002  0.08611655] [0.4300466  0.38758907 0.08541979]
0.3516961

0.8579309985379684 [0.32825866 0.40233098 0.11099457] [0.32794379 0.40382406 0.1121419 ]
0.5165026168999808 [0.4285435  0.38679034 0.08613262] [0.4300466  0.38758907 0.08541979]
0.42128838186241135 [0.07779254 0.06716514 0.26984604] [0.07838997 0.06767974 0.26930566]
1.0171069056133821 [0.15358365 0.22632508 0.10159528] [0.15446194 0.22566953 0.10086345]
1.0212751624175846 [0.18176276 0.1161001  0.06341994] [0.18233081 0.11583725 0.06251349]
0.2772593240043965 [0.53419652 0.55095952 0.09876192] [0.53396592 0.55006    0.09843945]
0.5649087443604299 [0.26558966 0.18807443 0.29205213] [0.26515203 0.18846833 0.29255763]
0.5599953263513655 [0.14911363 0.20911347 0.37200335] [0.14866517 0.2096845  0.37270436]
0.6501921637128938 [0.81890066 0.86660384 0.88882567] [0.81947767 0.86742554 0.89527595]
0.4305887671270058 [0.54377824 0.57311813 0.61440194] [0.54371524 0.57331565 0.6115708 ]
1.0059633355913478 [0.33997844 0.3571534  0.38609484] [0.33934504 0.35712131 0.38062272]
1.2106977360136228 [

0.5032364683962464 [0.14900025 0.20910051 0.372638  ] [0.14866517 0.2096845  0.37270436]
0.23326530064587347 [0.54294099 0.5722412  0.60906204] [0.54371524 0.57331565 0.6115708 ]
0.5356370963204714 [0.33948123 0.3566215  0.38278041] [0.33934504 0.35712131 0.38062272]
0.8332946722248896 [0.19099219 0.20068091 0.22212844] [0.19071851 0.20080176 0.21902172]
0.7829196994275074 [0.09396666 0.09855388 0.10803242] [0.093735   0.09857077 0.10598702]
0.4937515091132374 [0.02896563 0.03022847 0.03393827] [0.02887814 0.03020254 0.03315504]
Error on train: 0.7518123082551718
Error on test: 1.412758192898134
[ 19 ] ========
0.8288195752570372 [0.10956054 0.09981433 0.07215057] [0.10977999 0.09956808 0.07071454]
0.47083631793467967 [0.34574756 0.32733072 0.24462545] [0.34470537 0.32552718 0.24414595]
1.7168865825195274 [0.16678012 0.18542529 0.32591604] [0.16858397 0.18470163 0.32331183]
0.9573857627301962 [0.09931945 0.12131731 0.07024564] [0.09753446 0.11862227 0.06664057]
0.7703501357935134 [0.23